In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"


baseball_path = r'C:\Users\james\Documents\MLB\Database'

### Games

In [2]:
game_df = read_and_save_games(team_map, generate=False)

In [ ]:
# Set date?

In [3]:
date_list = list(game_df['date'].unique())

### A01. DraftKings

In [ ]:
%run "A01. DraftKings.ipynb"

##### 1. Contests

In [ ]:
# Scrape contests
contest_df = contests(todaysdate)
# To csv
contest_df.to_csv(os.path.join(baseball_path, 'A01. DraftKings', '1. Contests', f'Contests {todaysdate}.csv'), index=False)
# # To database
# contest_df.to_sql(f'Contests {todaysdate}', con=engine, index=False, if_exists='replace')

In [ ]:
# Select contests of interest
subset_df = contest_df[(contest_df['Name'].str.contains('Four-Seamer')) & (~contest_df['Name'].str.contains('Turbo|vs'))].reset_index(drop=True)
print(subset_df.to_string())

##### 2. Draftables

In [ ]:
# Loop over contests of interest
for i in range(len(subset_df)):
    # Extract draftGroupId
    draftGroupId = subset_df['draftGroupId'][i]
    try:
        # Scrape draftables (DK Salaries)
        draftable_df = draftables(draftGroupId)
        # To csv
        draftable_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), index=False, encoding='iso-8859-1')
        # To database
        # draftable_df.to_sql(f'Draftables {draftGroupId}', con=engine, index=False, if_exists='replace')
        print(f"Saved {subset_df['draftGroupId'][i]}")
    except:
        print(f"Didn't save {subset_df['draftGroupId'][i]}")

##### 3. Payouts

In [ ]:
# Loop over contests of interest
for i in range(len(subset_df)):
    # Extract contestKey
    contestKey = subset_df['contestKey'][i]
    # Scrape payouts
    payout_df = payouts(contestKey)
    # To csv
    payout_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "3. Payouts", f"Payouts {contestKey}.csv"), index=False, encoding='iso-8859-1')
    # # To database
    # payout_df.to_sql(f'Payouts {contestKey}', con=engine, index=False, if_exists='replace')

##### 4. Results

In [ ]:
# Read in yesterday's contest 
# yesterdays_contest_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '1. Contests', f'Contests {yesterdaysdate}.csv'))
sql_query = f'SELECT * FROM "Contests {yesterdaysdate}"'
yesterdays_contest_df = pd.read_sql_query(sql_query, con=engine)

# Select contests of interest
yesterdays_subset_df = yesterdays_contest_df[(yesterdays_contest_df['Name'].str.contains('Four-Seamer')) & (~yesterdays_contest_df['Name'].str.contains('Turbo|vs'))].reset_index(drop=True)
print(yesterdays_subset_df.to_string())

##### 5. Entry Results and 6. Player Results

In [ ]:
# Loop over yesterday's contests
for i in range(len(yesterdays_subset_df)):
    # Extract contestKey
    contestKey = yesterdays_subset_df['contestKey'][i]
    print(contestKey)
    # Scrape results
    results(contestKey)
    time.sleep(15)
    # Read into pandas
    results_df = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "4. Results", f"contest-standings-{contestKey}.csv"))
    
    # Entry results
    entry_results_df = entry_results(results_df)
    entry_results_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "5. Entry Results", f"Entry Results {contestKey}.csv"), index=False, encoding='iso-8859-1')
    # entry_results_df.to_sql(f'Entry Results {contestKey}', con=engine, index=False, if_exists='replace')
    # Player results
    player_results_df = player_results(results_df)
    player_results_df.to_csv(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results", f"Player Results {contestKey}.csv"), index=False, encoding='iso-8859-1')
    # player_results_df.to_sql(f'Player Results {contestKey}', con=engine, index=False, if_exists='replace')

### A02. MLB API

In [ ]:
%run "A02. MLB API.ipynb"

##### 1. Stats API 

In [ ]:
statsapi_df = plays_statsapi("03/20/2024", "11/15/2024")
statsapi_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "1. Stats API", "Stats API 2024.csv"), index=False, encoding='iso-8859-1')
# statsapi_df.astype('str').to_sql('Stats API 2024', con=engine, index=False, if_exists='replace')
del statsapi_df

##### 2. Statcast

In [ ]:
statcast_df = plays_statcast("2024-03-20", "2024-11-15")
statcast_df.to_csv(os.path.join(baseball_path, "A02. MLB API", "2. Statcast", "Statcast 2024.csv"), index=False, encoding='iso-8859-1')
# statcast_df.astype('str').to_sql('Statcast 2024', con=engine, index=False, if_exists='replace')
del statcast_df

##### 3. Dataset

In [ ]:
# %%time
# # think this can be cut, just create datasets in their respective files from the raw created above
# complete_dataset = create_pa_inputs(start_year=2015, end_year=2023, short=50, long=300)
# complete_dataset[model_inputs_plus].to_sql('Dataset', con=engine, index=False, if_exists='replace')
# # del complete_dataset

### A03. Steamer

In [ ]:
%run "A03. Steamer.ipynb"

In [ ]:
# Download Steamer weekly logs and save them to csv
download_logs()

In [ ]:
# Read in Steamer hitters 
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
# # To database
# steamer_hitters_df.to_sql('Steamer Hitters', con=engine, index=False, if_exists='replace')

# Read in Steamer pitchers
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
# # To database
# steamer_pitchers_df.to_sql('Steamer Pitchers', con=engine, index=False, if_exists='replace')

# Below this point needs to be run prior to simulating, may be best moved elsewhere (Morning, Evening files)

### Dates

In [ ]:
# # Date ranges
# start_date_str = "20230330"
# end_date_str = "20231101"

# # Convert to datetime 
# start_date = datetime.datetime.strptime(start_date_str, "%Y%m%d")
# end_date = datetime.datetime.strptime(end_date_str, "%Y%m%d")

# # Start with start date
# current_date = start_date

# # List of dates
# date_list = []
# # Loop over dates
# while current_date <= end_date:
#     formatted_date_str = current_date.strftime("%Y%m%d")
#     date_list.append(formatted_date_str)
#     current_date += datetime.timedelta(days=1)

### Games

In [ ]:
# # Read in today's games
# games = statsapi.schedule(start_date="10/01/2023", end_date="11/01/2023")

### A04. Bullpens

In [4]:
%run "A04. Bullpens.ipynb"

In [5]:
# Read in team map
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# Need this header to trick site into thinking this isn't a scrape
# Create bullpen dataframe (and csv)
# Note: can write to csv in parallel but not to sql
def bullpens(date, team_map, historic):    
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
        }

    # Create path
    if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {date}")):
        os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {date}"))
        time.sleep(1)
    else:
        print("Already exists")    

    # Scrape bullpens
    for i in range(len(team_map)):
        # Extract team's website URL
        mlburl = team_map['MLBURL'][i]
        # Extract team's Baseball Reference abbreviation
        bbrefteam = team_map['BBREFTEAM'][i]
        # try:
        # Scrape bullpens
        bullpen_df = scrape_bullpen(mlburl, header, bbrefteam, historic, date)
        # To csv
        bullpen_df.to_csv(os.path.join(baseball_path, "A04. Bullpens", f"Bullpens {date}", f"Bullpen {bbrefteam} {date}.csv"), index=False, encoding='iso-8859-1')
        # # To database
        # bullpen_df.to_sql(f'Bullpen {bbrefteam} {date}', con=engine, index=False, if_exists='replace')

        # return bullpen_df, bbrefteam, date

In [14]:
date_list = ['20240320', '20240321']
date_list = ['20240321']

In [15]:
# header = {
#       "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
#       "X-Requested-With": "XMLHttpRequest"
#         }

# historic = True

# for date in date_list:
#     # Create path
#     if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {date}")):
#         os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {date}"))
#         time.sleep(1)
#     else:
#         pass
    
#     for i in range(len(team_map)):
#         mlburl = team_map['MLBURL'][i]
#         bbrefteam = team_map['BBREFTEAM'][i]
#         if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A04. Bullpens', f"Bullpens {date}", f"Bullpen {bbrefteam} {date}.csv")):
#             # Scrape bullpens
#             try:
#                 bullpen_df = scrape_bullpen(mlburl, header, bbrefteam, historic, date)
#                 # To csv
#                 bullpen_df.to_csv(os.path.join(baseball_path, "A04. Bullpens", f"Bullpens {date}", f"Bullpen {bbrefteam} {date}.csv"), index=False, encoding='iso-8859-1')
#             except:
#                 print("Didn't work")
#         else:
#             pass

In [16]:
# Run in parallel
return_list = Parallel(n_jobs=-1, verbose=0)(delayed(bullpens)(date=date, team_map=team_map, historic=True) for date in date_list)

### A05. Rosters

In [17]:
%run "A05. Rosters.ipynb"

##### 1. Batting Orders

In [18]:
# Read in team map
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

In [19]:
def batting_orders(team_map, game_df, row):
    ### Extract info
    # Date
    date = game_df.loc[row]['date']
    # Game ID
    game_id = game_df.loc[row]['game_id']
    # Team IDs
    away_id = game_df.loc[row]['away_id']
    home_id = game_df.loc[row]['home_id']
    
    # Create path
    if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\1. Batting Orders', f"Batting Orders {date}")):
        os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters\1. Batting Orders', f"Batting Orders {date}"))
        time.sleep(1)


    # Loop over teams in matchup
    for team_id in [away_id, home_id]:
        if team_id == away_id:
            team = "away"
        else:
            team = "home"
        # Scrape away team batting order
        order_df = order(game_id, team_id, date, team)
        # Extract team name
        team_name = team_map.loc[team_map['teamId'] == team_id, 'BBREFTEAM'].values[0]
        # To csv
        order_df.to_csv(os.path.join(baseball_path, "A05. Rosters", "1. Batting Orders", f"Batting Orders {date}", f"Batting Order {team_name} {game_id}.csv"), index=False, encoding='iso-8859-1')
        # To database
        # order_df.to_sql(f'Batting Order {team} {game_id}', con=engine, index=False, if_exists='replace')

In [27]:
game_df = game_df.query('"20240320" <= date <= "20240321"').reset_index(drop=True)

In [28]:
# Run in parallel
return_list = Parallel(n_jobs=-1, verbose=0)(delayed(batting_orders)(team_map, game_df, row) for row in range(len(game_df)))      

##### 2. Rosters

In [29]:
def rosters(team_map, game_df, row):
    ### Extract info
    # Date
    date = game_df.loc[row]['date']
    # Game ID
    game_id = game_df.loc[row]['game_id']
    # Team IDs
    away_id = game_df.loc[row]['away_id']
    home_id = game_df.loc[row]['home_id']
    
    # Create path
    if not os.path.exists(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters', "2. Rosters", f"Rosters {date}")):
        os.makedirs(os.path.join(r'C:\Users\james\Documents\MLB\Database\A05. Rosters', "2. Rosters", f"Rosters {date}"))
        time.sleep(1)
    
    # Loop over teams in matchup
    for team_id in [away_id, home_id]:
        if team_id == away_id:
            team = "away"
        else:
            team = "home"
    
        # Extract team name
        bbrefteam = team_map.loc[team_map['teamId'] == team_id, 'BBREFTEAM'].values[0]
    
        # Scrape rosters
        roster_df = roster(team_id, date, "40Man")
        roster_df.drop_duplicates('id', inplace=True)
        # To csv
        roster_df.to_csv(os.path.join(baseball_path, "A05. Rosters", "2. Rosters", f"Rosters {date}", f"Roster {bbrefteam} {date}.csv"), index=False, encoding='iso-8859-1')
        # # To database
        # roster_df.to_sql(f'Roster {bbrefteam} {date}', con=engine, index=False, if_exists='replace')


In [30]:
# Run in parallel
return_list = Parallel(n_jobs=-1, verbose=0)(delayed(rosters)(team_map, game_df, row) for row in range(len(game_df)))      

### A06. Weather

In [33]:
%run "A06. Weather.ipynb"

In [50]:
todaysdate = "20240321"

##### 1. Swish Analytics

In [51]:
# Scrape Swish Analytics
swishanalytics_df = swishanalytics(todaysdate)
# To csv
swishanalytics_df.to_csv(os.path.join(baseball_path, "A06. Weather", "1. Swish Analytics", f"Swish Analytics {todaysdate}.csv"), index=False)
# # To database
# swishanalytics_df.to_sql(f'Swish Analytics {todaysdate}', con=engine, index=False, if_exists='replace')

KeyError: 'Start'

##### 2. RotoGrinders

In [52]:
# Scrape RotoGrinders
rotogrinders_df = rotogrinders(todaysdate)
# To csv
rotogrinders_df.to_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"), index=False)
# # To database
# rotogrinders_df.to_sql(f'RotoGrinders {todaysdate}', con=engine, index=False, if_exists='replace')

AttributeError: 'NoneType' object has no attribute 'find_all'

##### 3. Ballpark Pal

In [53]:
# Scrape Ballpark Pal
ballparkpal_df = ballparkpal(todaysdate)
# To csv
ballparkpal_df.to_csv(os.path.join(baseball_path, "A06. Weather", "3. Ballpark Pal", f"Ballpark Pal {todaysdate}.csv"), index=False)
# # To database
# ballparkpal_df.to_sql(f'Ballpark Pal {todaysdate}', con=engine, index=False, if_exists='replace')

### A07. Projections

In [54]:
%run "A07. Projections.ipynb"

##### 1. DFF

In [55]:
# Scrape slates
dff_slates_df = dff_slates(todaysdate)
# To csv
dff_slates_df.to_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "1. Slates", f"DFF Slates {todaysdate}.csv"), index=False)
# # To database
# dff_slates_df.to_sql(f'DFF Slates {todaysdate}', con=engine, index=False, if_exists='replace')

In [56]:
# Loop over slates
for code in dff_slates_df['URL']:
    # Scrape projections
    dff_projections_df = dff_projections(todaysdate, code)
    # To csv
    dff_projections_df.to_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "2. Projections", f"DFF Projections {code}.csv"), index=False)
    # # To database
    # dff_projections_df.to_sql(f'DFF Projections {code}', con=engine, index=False, if_exists='replace')

##### 2. RotoWire

In [57]:
# Scrape slates
roto_slates_df = roto_slates(todaysdate)
# To csv
roto_slates_df.to_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "1. Slates", f"RotoWire Slates {todaysdate}.csv"), index=False)
# # To database
# roto_slates_df.to_sql(f'RotoWire Slates {todaysdate}', con=engine, index=False, if_exists='replace')

In [58]:
# Loop over slates
for code in roto_slates_df['slateID']:
    # Scrape projections
    roto_projections_df = roto_projections(todaysdate, code)
    # To csv
    roto_projections_df.to_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "2. Projections", f"RotoWire Projections {code}.csv"), index=False)
    # # To database
    # roto_projections_df.to_sql(f'RotoWire Projections {code}', con=engine, index=False, if_exists='replace')    

### A08. Odds

In [59]:
%run "A08. Odds.ipynb"

In [60]:
# Scrape odds
odds_df = odds(todaysdate)
# To csv
odds_df.to_csv(os.path.join(baseball_path, "A08. Odds", f"Odds {todaysdate}.csv"), index=False)
# # To database
# odds_df.to_sql(f'Odds {todaysdate}', con=engine, index=False, if_exists='replace')    